### Importing libraries

In [1]:
import llama_index.core
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_parse import LlamaParse
import nest_asyncio; nest_asyncio.apply()

import json
import os
import re
from dotenv import load_dotenv
load_dotenv()

True

### Loading data and producing embeddings of it

In [2]:
PERSIST_DIR = "./storage"
LIST_OF_DOCS = ["raw_texts/"+f for f in os.listdir("Raw_texts")]

print(LIST_OF_DOCS)

['raw_texts/2-settext.pdf', 'raw_texts/3-settext.txt', 'raw_texts/3-slides.pdf', 'raw_texts/4-settext.pdf', 'raw_texts/4-slides.pdf', 'raw_texts/5-settext.pdf', 'raw_texts/9-settext.pdf']


#### Produce index with SimpleDirectoryReader

In [3]:
PERSIST_DIR = "./storage_SimpleDirectoryReader"
parser = "SDR"

if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("raw_texts").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

### Select the LLM

In [ ]:
model = "gpt-4o-mini"

Settings.llm = OpenAI(temperature=0.0, model="gpt-4o-mini")

In [4]:
model = "gpt-4o"

Settings.llm = OpenAI(temperature=0.0, model="gpt-4o")

In [5]:
query_engine = index.as_query_engine(OpenAI(model))

def pingGPT(text_input):
   response = query_engine.query(text_input)
   return response.response

In [6]:
# Open quizzes
LIST_OF_QUIZZES = ["JSON_quizzes_only_closed_questions/"+f for f in os.listdir("./JSON_quizzes_only_closed_questions") if f.endswith('.json')]
print(LIST_OF_QUIZZES)

['JSON_quizzes_only_closed_questions/itembank-2.json', 'JSON_quizzes_only_closed_questions/itembank-3-4.json', 'JSON_quizzes_only_closed_questions/itembank-5-6.json', 'JSON_quizzes_only_closed_questions/itembank-9a.json', 'JSON_quizzes_only_closed_questions/itembank-9b.json']


### Update the prompt for GPT to only respond with one of the options

In [7]:
from llama_index.core import PromptTemplate

new_prompt = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query. "
    "Only answer with the letter and text of the answer which you think is correct.\n"
    "Question: {query_str}\n"
    "Answer: "
)
new_tmpl = PromptTemplate(new_prompt)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": new_tmpl}
)

### Full loop

In [8]:
# Iterate over each question and get GPT's response
for pass_n in range(5):
    print(f"\nPass over the quizzes number {pass_n+1}")

    total_nquestions = 0
    total_ncorrects = 0
    itembank_reports = []

    for doc in LIST_OF_QUIZZES:
        with open(doc, 'r', encoding='UTF-8') as file:
            questions_JSON = json.load(file)
        
        itembank_number = re.search(r"\d[a|b]?(?:\-\d)?", doc).group()
        this_ncorrects = 0 # for correct answers in each separate itembank
        this_nquestions = 0

        #print(f"----------processing itembank-{itembank_number}----------")

        for question in questions_JSON:
            full_prompt = question['question'] + '\n' + '\n'.join(question['answers'])
            given_answer = pingGPT(full_prompt)  # Get the response from GPT
            question["GPT's response"] = given_answer  # Append the response to the question
            question["correct"] = given_answer==question["correct answer(s)"]

            this_nquestions+=1
            total_nquestions+=1
            if given_answer==question["correct answer(s)"]:
                total_ncorrects+=1
                this_ncorrects+=1
            
        itembank_reports.append(f"itembank-{itembank_number}:\t\tCorrect answers {this_ncorrects} / total questions {this_nquestions} * 100 = {round(this_ncorrects/this_nquestions*100, 2)}%")


        # Save the updated JSON data to a new file
        with open(f'./results/default_RAG_responses/default_RAG_{model}_responses-{itembank_number}.json', 'w') as outfile:
            json.dump(questions_JSON, outfile, indent=2)

    itembank_reports.append(f"\nOverall correct answers {total_ncorrects} / total questions {total_nquestions} * 100 = {round(total_ncorrects/total_nquestions*100, 2)}%\n")
    itembank_reports.append("------------------------------------------------------------------------\n\n")

    # save the prompt and number of correct questions
    with open(f"./results/default_rag_{model}_final_results.txt", 'a', encoding='UTF-8') as report_file:
        report_file.write('\n'.join(itembank_reports))


Pass over the quizzes number 1

Pass over the quizzes number 2

Pass over the quizzes number 3

Pass over the quizzes number 4

Pass over the quizzes number 5


### Observe with Phoenix (not necessary to run)

In [ ]:
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={os.getenv('PHOENIX_API_KEY')}"
llama_index.core.set_global_handler("arize_phoenix", endpoint="https://llamatrace.com/v1/traces")

with open('./JSON_questions_only_closed_questions/itembank-3-4.json', 'r', encoding='UTF-8') as file:
    questions_JSON = json.load(file)

for question in questions_JSON[18:19]:
    full_prompt = question['question'] + '\n' + '\n'.join(question['answers'])
    given_answer = pingGPT(full_prompt)  # Get the response from GPT
print(given_answer)